In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [10]:
%%capture
!pip install wandb

In [11]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

True

In [12]:
# Prepare the training dataset
BATCH_SIZE = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# build input pipeline using tf.data
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(BATCH_SIZE)

In [13]:
def Model():
    inputs = keras.Input(shape=(784,), name="digits")
    x1 = keras.layers.Dense(64, activation="relu")(inputs)
    x2 = keras.layers.Dense(64, activation="relu")(x1)
    outputs = keras.layers.Dense(10, name="predictions")(x2)

    return keras.Model(inputs=inputs, outputs=outputs)

def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

In [14]:
def train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          train_acc_metric,
          val_acc_metric,
          epochs=10, 
          log_step=200, 
          val_log_step=50):
  
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_states()
        val_acc_metric.reset_states()

        # log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

In [15]:
configs = {
              "learning_rate": 0.001,
              "epochs": 10,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
              "architecture": "CNN",
              "dataset": "CIFAR-10"
           }

run = wandb.init(project='my-tf-integration-dhiman', config=configs)
config = wandb.config

# initialize model
model = Model()

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      val_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.join()


Start of epoch 0
Training acc over epoch: 0.7628
Validation acc: 0.8489

Start of epoch 1
Training acc over epoch: 0.8660
Validation acc: 0.8811

Start of epoch 2
Training acc over epoch: 0.8915
Validation acc: 0.8959

Start of epoch 3
Training acc over epoch: 0.9063
Validation acc: 0.9061

Start of epoch 4
Training acc over epoch: 0.9159
Validation acc: 0.9088

Start of epoch 5
Training acc over epoch: 0.9225
Validation acc: 0.9165

Start of epoch 6
Training acc over epoch: 0.9277
Validation acc: 0.9161

Start of epoch 7
Training acc over epoch: 0.9317
Validation acc: 0.9197

Start of epoch 8
Training acc over epoch: 0.9351
Validation acc: 0.9191

Start of epoch 9
Training acc over epoch: 0.9381
Validation acc: 0.9231


epochs,9
loss,0.20991
acc,0.93813
val_loss,0.2989
val_acc,0.9231
_runtime,52
_timestamp,1622269714
_step,9


epochs,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▁▁▁▁▁▁
acc,▁▅▆▇▇▇████
val_loss,█▅▄▃▂▂▂▁▁▁
val_acc,▁▄▅▆▇▇▇███
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


In [16]:
def sweep_train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'batch_size': 8,
        'learning_rate': 0.01
    }

    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 7
    wandb.config.log_step = 200
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = "CNN"
    wandb.config.dataset_name = "CIFAR-10"

    # build input pipeline using tf.data
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(wandb.config.batch_size)

    val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    val_dataset = val_dataset.batch(wandb.config.batch_size)

    # initialize model
    model = Model()

    # Instantiate an optimizer to train the model.
    optimizer = keras.optimizers.SGD(learning_rate=wandb.config.learning_rate)
    # Instantiate a loss function.
    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Prepare the metrics.
    train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

    train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=wandb.config.epochs, 
          log_step=wandb.config.log_step, 
          val_log_step=wandb.config.val_log_step)

In [17]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [8, 16, 32, 64, 128, 256]
      },
      'learning_rate':{
          'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
      }
  }
}

In [18]:
sweep_id = wandb.sweep(sweep_config, project="my-tf-integration-dhiman")
wandb.agent(sweep_id, function=sweep_train)

Create sweep with ID: 56dhvmt4
Sweep URL: https://wandb.ai/dbhadore/my-tf-integration-dhiman/sweeps/56dhvmt4


wandb: Agent Starting Run: 6rpdxi5t with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 0.01


wandb: ERROR Run 6rpdxi5t errored: TypeError("train() got multiple values for argument 'epochs'")
wandb: Agent Starting Run: 99xs4c6s with config:
wandb: 	batch_size: 8
wandb: 	learning_rate: 0.0001


wandb: ERROR Run 99xs4c6s errored: TypeError("train() got multiple values for argument 'epochs'")
wandb: Agent Starting Run: um7f44x8 with config:
wandb: 	batch_size: 256
wandb: 	learning_rate: 0.01


wandb: ERROR Run um7f44x8 errored: TypeError("train() got multiple values for argument 'epochs'")
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true
